In [56]:
# general purpose modules for handling data
import numpy as np
from numpy import array
import pandas as pd

# for loading telo data column containing individual
# telomere length values
from ast import literal_eval

# custom module for handling telomere length data
import telomere_methods_astros as telo_ma

In [57]:
import importlib
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


---
&nbsp; 

The dataframe containing our telomere length data already contains the means. However, we're interested in analyzing (graphing, stats) the individual telomere length measurements. To do so we must extract the values from the list, while keeping the values associated with their timepont/individual. We'll achieve this by 'exploding' the cell containing individual telomere length measurements as a list, into one row per measurement (5520 rows per sample, 184 telos per 30 metaphases per sample). 

As well, we're interested in defining and examining the prevalence of short/long telomeres; this is achieved by finding the values that divide the data into bottom 25% (short telos), middle 50%, and top 25% (long telos). More on that soon.
 
&nbsp;
___

# Transforming telomere length data (teloFISH)

In [58]:
astro_df = pd.read_csv('../data/compiled data/All_astronauts_telomere_length_dataframe.csv')

# literal eval enables interpretation of individual telomere length values in the list 
astro_df['telo data'] = astro_df['telo data'].apply(lambda row: np.array(literal_eval(row)))

In [59]:
astro_df.head(4)

,astro number,astro id,timepoint,flight status,telo data,telo means,Q1,Q2-3,Q4
0,1,1536,L-270,Pre-Flight,"[138.33945873705312, 216.93952555963915, 128.0...",142.809227,telos preF Q1 <0.25,telos preF Q2-3 >0.25 & <0.75,telos preF Q4 >0.75
1,1,1536,L-180,Pre-Flight,"[43.56832609421985, 161.8610090210491, 182.358...",123.155724,telos preF Q1 <0.25,telos preF Q2-3 >0.25 & <0.75,telos preF Q4 >0.75
2,1,1536,L-60,Pre-Flight,"[94.4537253591714, 73.18743735382559, 128.0487...",122.126296,telos preF Q1 <0.25,telos preF Q2-3 >0.25 & <0.75,telos preF Q4 >0.75
3,1,1536,FD90,Mid-Flight,"[139.1413297694621, 120.83194119612429, 145.05...",157.284754,telos preF Q1 <0.25,telos preF Q2-3 >0.25 & <0.75,telos preF Q4 >0.75


## Exploding list containing individual telomere length measurements into rows

this action explodes the list into columns bearing each datapoint.
picture the list, which contains the individual values, expanding to the
right, up to 5520 measurements

importantly, the index #s per row still refer to the timepont/sample
so we can merge these columns back to the original astro_df
then we'll melt the columns into one, resulting in a lot of rows
where each has one individual telomere length measurement

In [60]:
explode_telos_raw = astro_df['telo data'].apply(pd.Series)

In [61]:
explode_telos_raw.head(2)

,0,1,2,3,4,5,6,7,8,9,...,5510,5511,5512,5513,5514,5515,5516,5517,5518,5519
0,138.339459,216.939526,128.065486,160.992315,32.141664,188.940862,168.042098,109.371868,136.251253,183.344470,...,132.191781,31.440027,164.951554,236.167725,89.976612,113.815570,194.587371,145.405947,45.456064,162.712997
1,43.568326,161.861009,182.358837,78.516539,81.506849,155.947210,89.475443,96.642165,96.842633,141.146007,...,61.326428,170.998998,75.893752,45.339125,159.037755,191.747411,172.134982,94.336786,145.172068,94.687604


In [62]:
exploded_telos_astro_df = (explode_telos_raw
    # merge 5520 columns with original dataframe
    .merge(astro_df, right_index = True, left_index = True)
                           
    # drop unnecessary columns
    .drop(['telo data', 'Q1', 'Q2-3', 'Q4'], axis = 1)
                           
    #specify which columns remain constant per indviidual telo 
    .melt(id_vars = ['astro number', 'astro id', 'timepoint', 'flight status', 'telo means'], value_name = "telo data exploded") 
    .drop("variable", axis = 1)
    .dropna())

exploded_telos_astro_df.shape

(436080, 6)

In [63]:
copy_exploded_telos_astros_df = exploded_telos_astro_df.copy()

copy_exploded_telos_astros_df.to_csv('../data/compiled and processed data/exploded_telos_astros_df.csv', index = False)

## Loading astro telomere length data per cell, exploding cell data, saving to file

In [64]:
astro_cells_df = pd.read_csv('../data/compiled data/All_astronauts_telomere_length_per_cell_dataframe.csv')

astro_cells_df['telo data per cell'] = astro_cells_df['telo data per cell'].apply(lambda row: np.array(literal_eval(row)))
astro_cells_df['telo data per cell'][0].shape

(30,)

In [65]:
explode_cells_raw = astro_cells_df['telo data per cell'].apply(pd.Series)

In [66]:
exploded_cells_astro_df = (explode_cells_raw

    .merge(astro_cells_df, right_index = True, left_index = True)
    .drop(['telo data per cell'], axis = 1)
    .melt(id_vars = ['astro number', 'astro id', 'timepoint', 'flight status', 'telo means'], value_name = "telo data per cell") 
    .drop("variable", axis = 1)
    .dropna())

# exploded_cells_astro_df[
#     (exploded_cells_astro_df['astro id'] == 5163) &
#     (exploded_cells_astro_df['timepoint'] == 'L-270')].shape

In [67]:
exploded_cells_astro_df.to_csv('../data/compiled and processed data/exploded_cells_astros_df.csv', index = False)

## Loading astro controls telomere length data per cell, exploding cell data, saving to file

In [68]:
ctrl_cells_df = pd.read_csv('../data/compiled data/All_controls_telomere_length_per_cell_dataframe.csv')

ctrl_cells_df['telo data per cell'] = ctrl_cells_df['telo data per cell'].apply(lambda row: np.array(literal_eval(row)))
ctrl_cells_df['telo data per cell'][0].shape

(30,)

In [69]:
explode_cells_ctrl_raw = ctrl_cells_df['telo data per cell'].apply(pd.Series)
ctrl_cells_df.head(4)

,control id,timepoint,flight status controls,telo data per cell,telo means
0,100,L-270,Pre-Flight,"[0.8477957561098561, 1.0075334075822138, 0.858...",1.013536
1,100,L-180,Pre-Flight,"[1.3042379742777852, 1.2996318440034278, 1.366...",1.268621
2,100,L-60,Pre-Flight,"[1.1800698226000297, 1.0767199987117924, 1.087...",0.909948
3,100,R+60,Post-Flight,"[1.5899987895105754, 1.0877994067201529, 1.071...",1.321577


In [71]:
exploded_cells_ctrl_df = (explode_cells_ctrl_raw

    .merge(ctrl_cells_df, right_index = True, left_index = True)
    .drop(['telo data per cell'], axis = 1)
    .melt(id_vars = ['control id', 'timepoint', 'flight status controls', 'telo means'], value_name = "telo data per cell") 
    .drop("variable", axis = 1)
    .dropna())

# exploded_cells_ctrl_df

In [72]:
exploded_cells_ctrl_df['flight status'] = 'Controls'

In [73]:
exploded_cells_ctrl_df.to_csv('../data/compiled and processed data/exploded_cells_ctrl_df.csv', index = False)

## Defining short/mid/long telomeres w/ quartiles and counting per timepoint per astro

This function identifies the timepoint per astronaut most distal to spaceflight (L-270 or L-180) and identifies the individual telomere length values which divide the data into the bottom 25% (short telos), mid 50%, and top 25% (long telos). The function then counts how many telos reside in each quartile. Now we have a way to define short/long telomeres.

The function then applies those quartile cutoff values to subsequent datapoints and counts how many telomeres reside within each quartile. In doing so, we count the number of telomeres moving into/out of the quartiles, per timepoint per astronaut, and can quantitatively dicuss # of short/long telos.

In [74]:
astro_df.head()

,astro number,astro id,timepoint,flight status,telo data,telo means,Q1,Q2-3,Q4
0,1,1536,L-270,Pre-Flight,"[138.33945873705312, 216.93952555963915, 128.0...",142.809227,telos preF Q1 <0.25,telos preF Q2-3 >0.25 & <0.75,telos preF Q4 >0.75
1,1,1536,L-180,Pre-Flight,"[43.56832609421985, 161.8610090210491, 182.358...",123.155724,telos preF Q1 <0.25,telos preF Q2-3 >0.25 & <0.75,telos preF Q4 >0.75
2,1,1536,L-60,Pre-Flight,"[94.4537253591714, 73.18743735382559, 128.0487...",122.126296,telos preF Q1 <0.25,telos preF Q2-3 >0.25 & <0.75,telos preF Q4 >0.75
3,1,1536,FD90,Mid-Flight,"[139.1413297694621, 120.83194119612429, 145.05...",157.284754,telos preF Q1 <0.25,telos preF Q2-3 >0.25 & <0.75,telos preF Q4 >0.75
4,1,1536,FD140,Mid-Flight,"[123.6050785165386, 183.1774139659205, 154.360...",187.027515,telos preF Q1 <0.25,telos preF Q2-3 >0.25 & <0.75,telos preF Q4 >0.75


In [75]:
quartiles_astro_df = telo_ma.make_quartiles_columns(astro_df)

In [76]:
for col in ['Q1', 'Q2-3', 'Q4']:
    quartiles_astro_df[col] = quartiles_astro_df[col].astype('int64')

quartiles_astro_df[quartiles_astro_df['astro id'] == 2171]

,astro number,astro id,timepoint,flight status,telo data,telo means,Q1,Q2-3,Q4
22,4,2171,L-180,Pre-Flight,"[150.43478260869566, 114.1863354037267, 72.981...",127.863014,1381,2759,1380
23,4,2171,L-60,Pre-Flight,"[173.1801242236025, 148.17391304347825, 183.70...",113.106425,1614,3235,671
24,4,2171,FD45,Mid-Flight,"[149.86335403726707, 121.06832298136646, 150.1...",135.290127,1413,2112,1995
25,4,2171,FD260,Mid-Flight,"[68.2360248447205, 109.98757763975155, 153.962...",134.600340,1230,2663,1627
26,4,2171,R+105,Post-Flight,"[105.27950310559007, 238.8944099378882, 98.149...",120.721879,1793,2600,1127
27,4,2171,R+180,Post-Flight,"[144.6335403726708, 110.77018633540372, 157.73...",112.335962,1558,3379,583
28,4,2171,R+270,Post-Flight,"[177.9751552795031, 118.88198757763975, 72.447...",96.448582,2498,2837,185


In [77]:
melted_quartiles_astro_df = pd.melt(
    quartiles_astro_df,
    id_vars = [col for col in quartiles_astro_df.columns if col != 'Q1' and col != 'Q2-3' and col != 'Q4'],
    var_name='relative Q',
    value_name='Q freq counts')

melted_quartiles_astro_df['Q freq counts'] = melted_quartiles_astro_df['Q freq counts'].astype('int64')
melted_quartiles_astro_df['astro id'] = melted_quartiles_astro_df['astro id'].astype('str')

In [78]:
melted_quartiles_astro_df['astro id'].unique()

array(['1536', '7673', '2479', '2171', '1261', '3228', '4819', '1062',
       '2494', '2381', '5163'], dtype=object)

In [79]:
copy_melted_quartiles_astro_df = melted_quartiles_astro_df
copy_melted_quartiles_astro_df['telo data'] = copy_melted_quartiles_astro_df['telo data'].apply(lambda row: row.tolist())

melted_quartiles_astro_df.to_csv('../data/compiled and processed data/melted_quartiles_astro_df.csv', index = False)

# Cleaning astronaut urine biochem data (n=11)

In [80]:
urine_biochem_data = pd.read_csv('../data/compiled data/urine_biochem_data.csv')
urine_biochem_data

,biochemistry,Pre,FD15,FD30,FD60,FD120,FD180,R+0 day 1,R+0 day 2,R+30 day 1,R+30 day 2,sample type
0,"8-OHdG, Urine ug/gCr",2.6 ± 1,3.1 ± 1,3 ± 1,3.6 ± 1.2,3 ± 1,2.5 ± 0.6,2.4 ± 0.8,2.4 ± 1,1.7 ± 0.7,1.8 ± 1,urine
1,"Copper, Urine umol/day",0.28 ± 0.17,0.24 ± 0.04,0.19 ± 0.02,0.24 ± 0.04,0.21 ± 0.05,0.24 ± 0.08,0.58 ± 0.71,0.25 ± 0.07,0.26 ± 0.1,0.34 ± 0.32,urine
2,PGF2-alpha ng/mg Cr,1.84 ± 1.2,2.22 ± 0.94,2.48 ± 1.05,2.92 ± 1.38,2.7 ± 1.26,3.12 ± 1.14,1.53 ± 1.06,1.59 ± 0.91,1.88 ± 1.12,1.72 ± 1.11,urine
3,"Selenium, Urine umol/day",1.02 ± 0.39,1.11 ± 0.35,1.08 ± 0.3,1.26 ± 0.54,1.14 ± 0.42,1.19 ± 0.42,0.88 ± 0.21,0.94 ± 0.27,0.96 ± 0.31,1.03 ± 0.41,urine


In [81]:
melt_urine_biochem_data = pd.melt(
    urine_biochem_data,
    id_vars=['biochemistry', 'sample type'],
    var_name='timepoint',
    value_name='measurement'
)

melt_urine_biochem_data.head(4)

,biochemistry,sample type,timepoint,measurement
0,"8-OHdG, Urine ug/gCr",urine,Pre,2.6 ± 1
1,"Copper, Urine umol/day",urine,Pre,0.28 ± 0.17
2,PGF2-alpha ng/mg Cr,urine,Pre,1.84 ± 1.2
3,"Selenium, Urine umol/day",urine,Pre,1.02 ± 0.39


In [82]:
def make_flight_status_via_timepoint(row):
    if 'Pre' in row:
        return 'Pre-Flight'
    elif 'L' in row:
        return 'Pre-Flight'
    elif 'FD' in row:
        return 'Mid-Flight'
    elif 'R' in row:
        return 'Post-Flight'

In [83]:
def grab_number_remove_plusminus(row):
    if '±' in str(row):
        row = row.split('±')[0].strip()
        row = float(row)
        return row  

In [84]:
melt_urine_biochem_data['flight status'] = melt_urine_biochem_data['timepoint'].apply(lambda row: make_flight_status_via_timepoint(row))

In [85]:
melt_urine_biochem_data['measured analyte'] = melt_urine_biochem_data['measurement'].apply(lambda row: grab_number_remove_plusminus(row))
melt_urine_biochem_data.head(4)

,biochemistry,sample type,timepoint,measurement,flight status,measured analyte
0,"8-OHdG, Urine ug/gCr",urine,Pre,2.6 ± 1,Pre-Flight,2.60
1,"Copper, Urine umol/day",urine,Pre,0.28 ± 0.17,Pre-Flight,0.28
2,PGF2-alpha ng/mg Cr,urine,Pre,1.84 ± 1.2,Pre-Flight,1.84
3,"Selenium, Urine umol/day",urine,Pre,1.02 ± 0.39,Pre-Flight,1.02


# Reading in astronaut blood biochem data (n=11)

In [86]:
blood_biochem_data = pd.read_csv('../data/compiled data/blood_biochem_data.csv')
blood_biochem_data.head(6)

,biochemistry,Pre,FD15,FD30,FD60,FD120,FD180,R+0,R+180,sample type
0,CCL2/MCP-1 pg/ml,83 ± 17,89 ± 21,94 ± 17,96 ± 26,96 ± 25,93 ± 22,182 ± 146,87 ± 23,blood
1,CCL3/MIP-1a pg/ml,438 ± 194,644 ± 214,652 ± 129,642 ± 218,489 ± 89,641 ± 247,310 ± 96,388 ± 105,blood
2,CCL4/MIP1B pg/ml,54 ± 17,75 ± 27,101 ± 34,75 ± 34,75 ± 37,70 ± 28,51 ± 16,48 ± 14,blood
3,CCL5/RANTES pg/ml,6902 ± 4393,15462 ± 2388,17986 ± 601,15022 ± 2726,14227 ± 2755,13636 ± 3964,3890 ± 3563,5569 ± 2539,blood
4,CXCL5/ENA-78 pg/ml,275 ± 549,1233 ± 967,1005 ± 426,1144 ± 823,1009 ± 503,919 ± 501,93 ± 74,134 ± 70,blood
5,CXCL8/IL-8 pg/ml,12 ± 5,23 ± 17,14 ± 7,18 ± 6,12 ± 2,19 ± 9,*,*,blood


In [87]:
blood_biochem_data.replace('*', np.NaN, inplace=True)

In [88]:
melt_blood_biochem_data = pd.melt(
    blood_biochem_data,
    id_vars=['biochemistry', 'sample type'],
    var_name='timepoint',
    value_name='measurement')

In [89]:
melt_blood_biochem_data['flight status'] = melt_blood_biochem_data['timepoint'].apply(lambda row: make_flight_status_via_timepoint(row))
melt_blood_biochem_data['measured analyte'] = melt_blood_biochem_data['measurement'].apply(lambda row: grab_number_remove_plusminus(row))
melt_blood_biochem_data.head(4)

,biochemistry,sample type,timepoint,measurement,flight status,measured analyte
0,CCL2/MCP-1 pg/ml,blood,Pre,83 ± 17,Pre-Flight,83.0
1,CCL3/MIP-1a pg/ml,blood,Pre,438 ± 194,Pre-Flight,438.0
2,CCL4/MIP1B pg/ml,blood,Pre,54 ± 17,Pre-Flight,54.0
3,CCL5/RANTES pg/ml,blood,Pre,6902 ± 4393,Pre-Flight,6902.0


In [90]:
melt_blood_biochem_data.rename(columns={'biochemistry':'biochemistry analyte'}, inplace=True)
melt_urine_biochem_data.rename(columns={'biochemistry':'biochemistry analyte'}, inplace=True)

In [91]:
melt_urine_biochem_data.to_csv('../data/compiled and processed data/melt_urine_biochem_data.csv', index=False)
melt_blood_biochem_data.to_csv('../data/compiled and processed data/melt_blood_biochem_data.csv', index=False)